# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 10:48AM,254160,16,EMERSA-8984103,"Emersa Waterbox, LLC",Released
1,Jan 3 2023 10:46AM,254159,22,637546,"NBTY Global, Inc.",Released
2,Jan 3 2023 10:39AM,254155,10,0086271943,ISDIN Corporation,Released
3,Jan 3 2023 10:39AM,254155,10,0086271993,ISDIN Corporation,Released
4,Jan 3 2023 10:39AM,254155,10,0086271998,ISDIN Corporation,Released
5,Jan 3 2023 10:39AM,254155,10,0086272179,ISDIN Corporation,Released
6,Jan 3 2023 10:39AM,254155,10,0086272217,ISDIN Corporation,Released
7,Jan 3 2023 10:39AM,254155,10,0086272218,ISDIN Corporation,Released
8,Jan 3 2023 10:39AM,254155,10,0086272216,ISDIN Corporation,Released
9,Jan 3 2023 10:39AM,254155,10,0086272256,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,254155,Released,47
44,254156,Released,7
45,254158,Released,1
46,254159,Released,1
47,254160,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254155,NaN,NaN,47.0
254156,NaN,NaN,7.0
254158,NaN,NaN,1.0
254159,NaN,NaN,1.0
254160,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,18.0,33.0,11.0
254104,0.0,0.0,1.0
254108,0.0,0.0,24.0
254111,0.0,0.0,53.0
254113,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,18,33,11
254104,0,0,1
254108,0,0,24
254111,0,0,53
254113,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,18,33,11
1,254104,0,0,1
2,254108,0,0,24
3,254111,0,0,53
4,254113,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,18,33,11
1,254104,,,1
2,254108,,,24
3,254111,,,53
4,254113,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 10:48AM,254160,16,"Emersa Waterbox, LLC"
1,Jan 3 2023 10:46AM,254159,22,"NBTY Global, Inc."
2,Jan 3 2023 10:39AM,254155,10,ISDIN Corporation
49,Jan 3 2023 10:38AM,254158,10,Eye Pharma Inc
50,Jan 3 2023 10:34AM,254154,10,ISDIN Corporation
76,Jan 3 2023 10:34AM,254150,10,ISDIN Corporation
104,Jan 3 2023 10:33AM,254153,10,ISDIN Corporation
108,Jan 3 2023 10:33AM,254151,10,ISDIN Corporation
110,Jan 3 2023 10:30AM,254156,10,ISDIN Corporation
117,Jan 3 2023 10:26AM,254146,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 3 2023 10:48AM,254160,16,"Emersa Waterbox, LLC",,,1
1,Jan 3 2023 10:46AM,254159,22,"NBTY Global, Inc.",,,1
2,Jan 3 2023 10:39AM,254155,10,ISDIN Corporation,,,47
3,Jan 3 2023 10:38AM,254158,10,Eye Pharma Inc,,,1
4,Jan 3 2023 10:34AM,254154,10,ISDIN Corporation,,,26
5,Jan 3 2023 10:34AM,254150,10,ISDIN Corporation,,,28
6,Jan 3 2023 10:33AM,254153,10,ISDIN Corporation,,,4
7,Jan 3 2023 10:33AM,254151,10,ISDIN Corporation,,,2
8,Jan 3 2023 10:30AM,254156,10,ISDIN Corporation,,,7
9,Jan 3 2023 10:26AM,254146,10,ISDIN Corporation,,,31


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 10:48AM,254160,16,"Emersa Waterbox, LLC",1,,
1,Jan 3 2023 10:46AM,254159,22,"NBTY Global, Inc.",1,,
2,Jan 3 2023 10:39AM,254155,10,ISDIN Corporation,47,,
3,Jan 3 2023 10:38AM,254158,10,Eye Pharma Inc,1,,
4,Jan 3 2023 10:34AM,254154,10,ISDIN Corporation,26,,
5,Jan 3 2023 10:34AM,254150,10,ISDIN Corporation,28,,
6,Jan 3 2023 10:33AM,254153,10,ISDIN Corporation,4,,
7,Jan 3 2023 10:33AM,254151,10,ISDIN Corporation,2,,
8,Jan 3 2023 10:30AM,254156,10,ISDIN Corporation,7,,
9,Jan 3 2023 10:26AM,254146,10,ISDIN Corporation,31,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 10:48AM,254160,16,"Emersa Waterbox, LLC",1,,
1,Jan 3 2023 10:46AM,254159,22,"NBTY Global, Inc.",1,,
2,Jan 3 2023 10:39AM,254155,10,ISDIN Corporation,47,,
3,Jan 3 2023 10:38AM,254158,10,Eye Pharma Inc,1,,
4,Jan 3 2023 10:34AM,254154,10,ISDIN Corporation,26,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 10:48AM,254160,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Jan 3 2023 10:46AM,254159,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jan 3 2023 10:39AM,254155,10,ISDIN Corporation,47.0,NaN,NaN
3,Jan 3 2023 10:38AM,254158,10,Eye Pharma Inc,1.0,NaN,NaN
4,Jan 3 2023 10:34AM,254154,10,ISDIN Corporation,26.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 10:48AM,254160,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Jan 3 2023 10:46AM,254159,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jan 3 2023 10:39AM,254155,10,ISDIN Corporation,47.0,0.0,0.0
3,Jan 3 2023 10:38AM,254158,10,Eye Pharma Inc,1.0,0.0,0.0
4,Jan 3 2023 10:34AM,254154,10,ISDIN Corporation,26.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7624064,871.0,3.0,0.0
12,762362,3.0,0.0,0.0
15,762381,41.0,40.0,0.0
16,254160,1.0,0.0,0.0
19,762362,15.0,33.0,18.0
20,508253,2.0,0.0,0.0
21,508286,1.0,1.0,0.0
22,254159,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7624064,871.0,3.0,0.0
1,12,762362,3.0,0.0,0.0
2,15,762381,41.0,40.0,0.0
3,16,254160,1.0,0.0,0.0
4,19,762362,15.0,33.0,18.0
5,20,508253,2.0,0.0,0.0
6,21,508286,1.0,1.0,0.0
7,22,254159,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,871.0,3.0,0.0
1,12,3.0,0.0,0.0
2,15,41.0,40.0,0.0
3,16,1.0,0.0,0.0
4,19,15.0,33.0,18.0
5,20,2.0,0.0,0.0
6,21,1.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,871.0
1,12,Released,3.0
2,15,Released,41.0
3,16,Released,1.0
4,19,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
Executing,3.0,0.0,40.0,0.0,33.0,0.0,1.0,0.0
Released,871.0,3.0,41.0,1.0,15.0,2.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
1,Executing,3.0,0.0,40.0,0.0,33.0,0.0,1.0,0.0
2,Released,871.0,3.0,41.0,1.0,15.0,2.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
1,Executing,3.0,0.0,40.0,0.0,33.0,0.0,1.0,0.0
2,Released,871.0,3.0,41.0,1.0,15.0,2.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()